# Estimating Tour Mode Choice

This notebook illustrates how to re-estimate tour and subtour mode choice for ActivitySim.  This process 
includes running ActivitySim in estimation mode to read household travel survey files and write out
the estimation data bundles used in this notebook.  To review how to do so, please visit the other
notebooks in this directory.

# Load libraries

In [1]:
import larch as lx
import pandas as pd

lx.versions()

JAX not found. Some functionality will be unavailable.


{'larch': '6.0.32',
 'sharrow': '2.13.0',
 'numpy': '1.26.4',
 'pandas': '1.5.3',
 'xarray': '2024.3.0',
 'numba': '0.60.0'}

For this demo, we will assume that you have already run ActivitySim in estimation
mode, and saved the required estimation data bundles (EDB's) to disk.  See
the [first notebook](./01_estimation_mode.ipynb) for details.  The following module
will run a script to set everything up if the example data is not already available.

In [2]:
from est_mode_setup import prepare

prepare()

EDB directory already populated.


PosixPath('test-estimation-data/activitysim-prototype-mtc-extended')

# Load data and prep model for estimation

In [3]:
modelname = "tour_mode_choice"

from activitysim.estimation.larch import component_model

model, data = component_model(
    modelname,
    edb_directory=f"output-est-mode/estimation_data_bundle/{modelname}/",
    return_data=True,
)

loading from output-est-mode/estimation_data_bundle/tour_mode_choice/tour_mode_choice_coefficients.csv
loading from output-est-mode/estimation_data_bundle/tour_mode_choice/tour_mode_choice_coefficients_template.csv
loading spec from output-est-mode/estimation_data_bundle/tour_mode_choice/tour_mode_choice_SPEC.csv
loading from output-est-mode/estimation_data_bundle/tour_mode_choice/tour_mode_choice_values_combined.parquet


The tour mode choice model is already a `ModelGroup` segmented on different purposes,
so we can add the subtour mode choice as just another member model of the group

In [4]:
model2, data2 = component_model(
    "atwork_subtour_mode_choice", 
    edb_directory="output-est-mode/estimation_data_bundle/atwork_subtour_mode_choice",
    return_data=True,
    )

loading from output-est-mode/estimation_data_bundle/atwork_subtour_mode_choice/tour_mode_choice_coefficients.csv
loading from output-est-mode/estimation_data_bundle/atwork_subtour_mode_choice/atwork_subtour_mode_choice_coefficients_template.csv
loading spec from output-est-mode/estimation_data_bundle/atwork_subtour_mode_choice/atwork_subtour_mode_choice_SPEC.csv
loading from output-est-mode/estimation_data_bundle/atwork_subtour_mode_choice/atwork_subtour_mode_choice_values_combined.parquet


In [5]:
model.extend(model2)

# Review data loaded from the EDB

The next step is to read the EDB, including the coefficients, model settings, utilities specification, and chooser and alternative data.

### Coefficients

In [6]:
data.coefficients

,value,constrain
coefficient_name,,
coef_one,1.000,T
coef_nest_root,1.000,T
coef_nest_AUTO,0.720,T
coef_nest_AUTO_DRIVEALONE,0.350,T
coef_nest_AUTO_SHAREDRIDE2,0.350,T
...,...,...
walk_transit_CBD_ASC_atwork,0.564,F
drive_transit_CBD_ASC_eatout_escort_othdiscr_othmaint_shopping_social,0.525,F
drive_transit_CBD_ASC_school_univ,0.672,F


#### Utility specification

In [7]:
data.spec

,Label,Description,Expression,DRIVEALONEFREE,DRIVEALONEPAY,SHARED2FREE,SHARED2PAY,SHARED3FREE,SHARED3PAY,WALK,...,WALK_HVY,WALK_COM,DRIVE_LOC,DRIVE_LRF,DRIVE_EXP,DRIVE_HVY,DRIVE_COM,TAXI,TNC_SINGLE,TNC_SHARED
0,util_DRIVEALONEFREE_Unavailable,DRIVEALONEFREE - Unavailable,sov_available == False,-999,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,util_DRIVEALONEFREE_Unavailable_for_zero_auto_...,DRIVEALONEFREE - Unavailable for zero auto hou...,auto_ownership == 0,-999,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,util_DRIVEALONEFREE_Unavailable_for_persons_le...,DRIVEALONEFREE - Unavailable for persons less ...,age < 16,-999,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,util_DRIVEALONEFREE_Unavailable_for_joint_tours,DRIVEALONEFREE - Unavailable for joint tours,is_joint == True,-999,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,util_DRIVEALONEFREE_Unavailable_if_didnt_drive...,DRIVEALONEFREE - Unavailable if didn't drive t...,is_atwork_subtour & ~work_tour_is_SOV,-999,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
310,util_Drive_to_Transit_dest_CBD,Drive to Transit dest CBD,@df.destination_in_cbd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,drive_transit_CBD_ASC,drive_transit_CBD_ASC,drive_transit_CBD_ASC,drive_transit_CBD_ASC,drive_transit_CBD_ASC,NaN,NaN,NaN
311,util_Drive_to_Transit_distance_penalty,Drive to Transit - distance penalty,@drvtrn_distpen_0_multiplier * (1-od_skims['DI...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,coef_ivt,coef_ivt,coef_ivt,coef_ivt,coef_ivt,NaN,NaN,NaN
312,util_Walk_not_available_for_long_distances,Walk not available for long distances,@od_skims.max('DISTWALK') > 3,NaN,NaN,NaN,NaN,NaN,NaN,-999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
313,util_Bike_not_available_for_long_distances,Bike not available for long distances,@od_skims.max('DISTBIKE') > 8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Chooser data

In [8]:
data.chooser_data

,model_choice,override_choice,util_DRIVEALONEFREE_Unavailable,util_DRIVEALONEFREE_Unavailable_for_zero_auto_households,util_DRIVEALONEFREE_Unavailable_for_persons_less_than_16,util_DRIVEALONEFREE_Unavailable_for_joint_tours,util_DRIVEALONEFREE_Unavailable_if_didnt_drive_to_work,util_DRIVEALONEFREE_In_vehicle_time,util_DRIVEALONEFREE_Terminal_time,util_DRIVEALONEFREE_Operating_cost,...,walk_lrf_available,walk_ferry_available,drive_local_available,drive_commuter_available,drive_express_available,drive_heavyrail_available,drive_lrf_available,drive_ferry_available,destination_in_cbd,override_choice_code
tour_id,,,,,,,,,,,,,,,,,,,,,
1870,SHARED3FREE,SHARED3FREE,0.0,0.0,0.0,0.0,0.0,20.830002,21.26572,6.293324,...,False,False,True,True,False,True,False,False,1,5
20468,SHARED3FREE,TNC_SINGLE,0.0,0.0,0.0,0.0,0.0,9.440000,13.24624,14.917349,...,False,False,True,True,False,True,False,False,0,20
27055,SHARED3FREE,SHARED3FREE,0.0,0.0,0.0,0.0,0.0,13.160000,8.53068,25.088453,...,False,False,True,True,False,True,False,False,0,5
38877,SHARED2FREE,SHARED3FREE,0.0,0.0,0.0,0.0,0.0,6.730000,11.04132,11.040045,...,False,False,True,True,False,True,False,False,0,5
38904,SHARED2FREE,SHARED2FREE,0.0,0.0,0.0,0.0,0.0,22.410000,17.29320,46.920191,...,True,False,True,True,False,True,True,False,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
310197956,DRIVEALONEFREE,DRIVEALONEFREE,0.0,0.0,0.0,0.0,0.0,29.900002,4.21092,69.245528,...,False,False,False,False,False,False,False,False,0,1
310202376,DRIVEALONEFREE,BIKE,0.0,0.0,0.0,0.0,0.0,5.480000,3.82392,19.277738,...,False,False,False,False,False,False,False,False,0,8
310202384,DRIVEALONEFREE,DRIVEALONEFREE,0.0,0.0,0.0,0.0,0.0,5.480000,3.82392,19.277738,...,False,False,False,False,False,False,False,False,0,1


# Estimate

With the model setup for estimation, the next step is to estimate the model coefficients.  Make sure to use a sufficiently large enough household sample and set of zones to avoid an over-specified model, which does not have a numerically stable likelihood maximizing solution.  Larch has a built-in estimation methods including BHHH, and also offers access to more advanced general purpose non-linear optimizers in the `scipy` package, including SLSQP, which allows for bounds and constraints on parameters.  BHHH is the default and typically runs faster, but does not follow constraints on parameters.

In [9]:
model.doctor(repair_ch_av="-")

(<larch.ModelGroup "None">,
 [(<larch.Model (GEV) "eatout">, dictx()),
  (<larch.Model (GEV) "escort">, dictx()),
  (<larch.Model (GEV) "othdiscr">, dictx()),
  (<larch.Model (GEV) "othmaint">, dictx()),
  (<larch.Model (GEV) "school">, dictx()),
  (<larch.Model (GEV) "shopping">, dictx()),
  (<larch.Model (GEV) "social">, dictx()),
  (<larch.Model (GEV) "univ">, dictx()),
  (<larch.Model (GEV) "work">, dictx()),
  (<larch.Model (GEV) "atwork">, dictx())])

In [10]:
result = model.maximize_loglike(method="SLSQP", options={"maxiter": 1000})

,value,best,initvalue,minimum,maximum,nullvalue,holdfast
param_name,,,,,,,
-999,-999.000000,-999.000000,-999.000000,-999.0,-999.0,0.0,1
1,1.000000,1.000000,1.000000,1.0,1.0,0.0,1
bike_ASC_auto_deficient_atwork,-0.997746,-0.997746,-0.807408,-inf,inf,0.0,0
bike_ASC_auto_deficient_eatout,-1.673695,-1.673695,-1.569111,-inf,inf,0.0,0
bike_ASC_auto_deficient_escort,-4.147119,-4.147119,-4.527928,-inf,inf,0.0,0
...,...,...,...,...,...,...,...
walk_transit_ASC_no_auto_work,4.665799,4.665799,5.035417,-inf,inf,0.0,0
walk_transit_CBD_ASC_atwork,1.146544,1.146544,0.564000,-inf,inf,0.0,0
walk_transit_CBD_ASC_eatout_escort_othdiscr_othmaint_shopping_social,1.074027,1.074027,0.525000,-inf,inf,0.0,0


/Users/jpn/Git/est-mode/larch/src/larch/model/optimization.py:338: UserWarning: SLSQP may not play nicely with unbounded parameters
if you get poor results, consider setting global bounds with model.set_cap()
  warnings.warn(  # infinite bounds #  )


In [11]:
model.calculate_parameter_covariance()

/var/folders/l1/yt63lf3n60b1dc25d_y2d1q80000gn/T/ipykernel_30646/3375301916.py:1: PossibleOverspecification: Model is possibly over-specified (hessian is nearly singular).
  model.calculate_parameter_covariance()


(array([0.00000000e+00, 0.00000000e+00, 3.35905128e-01, 4.03997542e-01,
        3.98391399e-01, 2.04116538e-01, 3.43847113e-01, 3.05273022e-01,
        2.20308907e-01, 3.19155920e-01, 3.56339406e-02, 7.75126393e-02,
        4.36635034e+02, 1.46654623e-01, 3.88297006e-01, 9.37023760e-02,
        1.79285943e-01, 1.04219523e-01, 1.29302923e-01, 2.01485594e-01,
        1.57876345e-02, 6.56232000e-02, 7.82486553e+01, 7.76646316e+01,
        7.76685862e+01, 7.76638395e+01, 7.76632845e+01, 1.06222057e+02,
        7.76632209e+01, 7.76636115e+01, 2.25096550e-02, 7.76634919e+01,
                   nan, 1.77157706e-01, 8.99952061e-02, 2.05472670e-01,
        4.24820891e-02, 7.46055946e-02, 8.07063927e-02, 6.81778880e-02,
        3.12545218e-02, 5.76135157e-02, 1.78374744e-02, 9.09277391e-02,
        4.79541094e-04, 1.44415038e-04, 3.21619072e-04, 1.47413838e-04,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.000000

### Estimated coefficients

In [12]:
model.parameter_summary()

,Value,Std Err,t Stat,Signif,Null Value
Parameter,,,,,
-999,-999.,0.00,NA,,0.00
1,1.00,0.00,NA,,0.00
bike_ASC_auto_deficient_atwork,-0.998,0.336,-2.97,**,0.00
bike_ASC_auto_deficient_eatout,-1.67,0.404,-4.14,***,0.00
bike_ASC_auto_deficient_escort,-4.15,0.398,-10.41,***,0.00
bike_ASC_auto_deficient_othdiscr,-1.11,0.204,-5.42,***,0.00
bike_ASC_auto_deficient_othmaint,-2.34,0.344,-6.81,***,0.00
bike_ASC_auto_deficient_school,-1.44,0.305,-4.71,***,0.00
bike_ASC_auto_deficient_shopping,-1.92,0.220,-8.74,***,0.00


# Output Estimation Results

In [13]:
from activitysim.estimation.larch import update_coefficients
result_dir = data.edb_directory/"estimated"
update_coefficients(
    model, data, result_dir,
    output_file=f"{modelname}_coefficients_revised.csv",
);

### Write the model estimation report, including coefficient t-statistic and log likelihood

In [14]:
model.to_xlsx(
    result_dir/f"{modelname}_model_estimation.xlsx", 
    data_statistics=False,
)

# Next Steps

The final step is to either manually or automatically copy the `*_coefficients_revised.csv` file to the configs folder, rename it to `*_coefficients.csv`, and run ActivitySim in simulation mode.

In [15]:
pd.read_csv(result_dir/f"{modelname}_coefficients_revised.csv")

,coefficient_name,value,constrain
0,coef_one,1.000000,T
1,coef_nest_root,1.000000,T
2,coef_nest_AUTO,0.720000,T
3,coef_nest_AUTO_DRIVEALONE,0.350000,T
4,coef_nest_AUTO_SHAREDRIDE2,0.350000,T
...,...,...,...
302,walk_transit_CBD_ASC_atwork,1.146544,F
303,drive_transit_CBD_ASC_eatout_escort_othdiscr_o...,1.407306,F
304,drive_transit_CBD_ASC_school_univ,1.179871,F
305,drive_transit_CBD_ASC_work,1.648397,F
